# **BERTopic 실습**

이번 노트북에서는 BERTopic 모델을 한국어에 맞춰 사용할 수 있도록 Customizing한 후 인퍼런스 할 예정입니다. 인퍼런스 능력을 향상시키기 위해, 직접 BERT모델을 고르고, Tokenizer을 수정하며 주제를 merge하는 작업을 수행합니다. 이후, 결과를 시계열에 맞춰 시각화한 후 토픽 모델링의 결과를 확인합니다.

## **1 - 준비작업**

### **1.1 - 라이브러리 설치**
필요한 라이브러리를 설치합니다. 별도로 수정하지 말아주세요.

In [1]:
!pip install \
  torch \
  torchvision \
  torchaudio \
  --extra-index-url https://download.pytorch.org/whl/cu116

# !pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
# !pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
# !pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
# !pip install --upgrade cupy-cuda12x -f https://pip.cupy.dev/aarch64
# !pip uninstall cupy-cuda11x --yes

!pip install \
  bertopic \
  bertopic[visualization]

!pip install \
  konlpy

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 61.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 20.5 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.8 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039281 sha256=3afbdc2dfb3cabdf4acfd78d6360dd815591685fdf67975be7d5910dd0d8eb7e
  Stored in directory: /root/.cache/pip/wheels/7

### **1.2 - 라이브러리 불러오기**
필요한 라이브러리를 불러옵니다. 별도로 수정하지 말아주세요.

In [2]:
import os
import csv
import numpy as np
import random
import torch
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# from umap import UMAP
# from hdbscan import HDBSCAN
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

from konlpy.tag import Kkma, Komoran

from typing import List, Tuple, Dict, Literal, Optional, Callable

### **1.3 - Seed 고정**
결과를 일정하게 유지하기 위해 모든 SEED를 고정합니다. 별도로 수정하지 말아주세요.

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

### **1.4 - Colab**
코랩에 접근하기 위해 실행합니다. 그 전에, Colab 폴더에 코드와 데이터셋을 올바르게 넣었는지 확인해주세요.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **2 - 데이터셋**

### **2.1 - 데이터셋 로드**
데이터셋을 드라이브에서 불러옵니다.

In [5]:
PATH = './datasets/원자력+원전+탈원전_news_df.xlsx'
df1 = pd.read_excel(PATH)

PATH = './datasets/사설_반대.xlsx'
df2 = pd.read_excel(PATH)

PATH = './datasets/고리+신한울+새울+월성+신월성_news_df.xlsx'
df3 = pd.read_excel(PATH)

# df = pd.concat([df1, df2, df3], axis=0)

df = df2

미리보기

In [6]:
df.head()

,news_title,news_date,news_company,news_body
0,"[사설] `탈원전`, 방향은 맞지만 부작용 최소화해야",2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,[사설] 탈원전·석탄 조급증…안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,[fn사설] 새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,[사설] 탈원전…정규직화…'밀어붙이기 정책'에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,<사설>성급한 脫원전은 電力안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


### **2.2 - 텍스트 전처리**
해당 데이터는 크롤링된 데이터로, 올바른 결과를 위해서는 전처리가 필요합니다.
뉴스 기사를 위한 전처리 코드는 https://colab.research.google.com/drive/1O42v6YzHbTo_l36VG1xGp0aEuA0J8M3V?usp=sharing#scrollTo=P7JBkNna1jze 에서 가져왔습니다.

In [7]:
df['news_title'] = df['news_title'].map(lambda x: str(x))

In [8]:
import re, unicodedata
from string import whitespace
pattern_whitespace = re.compile(f'[{whitespace}]+')

df['news_title'] = df['news_title'].str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()


In [9]:
def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result

df['news_title'] = df['news_title'].map(clean_byline)

In [10]:
from string import whitespace, punctuation

def text_filter(text):
    punct_except_percent = ''.join([chr for chr in punctuation if chr != '%'])
    whitespace_convert_pattern = re.compile(f'[{whitespace}{punct_except_percent}]+')
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣]+')
    result = whitespace_convert_pattern.sub(' ', text)
    result = exclude_pattern.sub(' ', result).strip()
    result = whitespace_convert_pattern.sub(' ', result)
    return result

df['news_title'] = df['news_title'].map(text_filter)

전처리 후 결과 미리보기

In [11]:
df.head()

,news_title,news_date,news_company,news_body
0,탈원전 방향은 맞지만 부작용 최소화해야,2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,탈원전 석탄 조급증 안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,탈원전 정규직화 밀어붙이기 정책 에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,성급한 원전은 안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


## **3 - 모델**

기본으로 제공하는 SBERT는 영어 문장을 대상으로만 문장을 분류, 군집화 할 수 있습니다. <br>
한국어를 활용해 SBERT를 수행하고 싶다면, 한국어를 지원하는 BERT모델을 사용해야합니다. <br>
한국어를 지원하는 SBERT모델은 [여기]()에서 찾을수 있습니다.<br>
(이때, 주의할 점은 문장간의 유사도를 판별할 수 있는 모델을 사용해야 하기 때문에, 꼭 일반적인 BERT가 아닌 SBERT를 사용해야 합니다!)

**HuggingFace에서 한국어 SBERT모델 찾는법** <br>
- Hugging Face > Models > Natural Language Processing(좌측 메뉴)에 들어간다
- Filter by name에 "ko"라고 검색
- 검색된 다양한 모델에 들어간 후 원하는 모델을 정합니다.

**모델을 고르는 방법** <br>
- 모델의 성능 (주관적이지만 기능을 수행하기에 무리가 없는 수준이어야 합니다.)
- 어떤 데이터셋으로 학습되었는가? (내가 사용할 Domain과 비슷한 Corpus로 학습된 모델이 좋습니다.)

실습은 Hugging Face의 "jhgan/ko-sroberta-multitask"을 사용해서 진행하겠습니다.

### **3.1 - 모델 불러오기**

그냥 모델을 간단히 불러와서 설정해도 되지만, 원하는 결과를 도출하기 위해서는 다양한 하이퍼파라미터를 직접 설정할 필요가 있습니다.
아래의 주석에 맞춰 차근차근 따라가보세요.

In [12]:
embedding_model = 'jhgan/ko-sroberta-multitask'

### **3.2 - 커스텀 토크나이저 설정**

BERTopic에서 사용하는 Vectorizer는 기본적으로 띄어쓰기를 기준으로 토큰화를 진행합니다. 우리는 영어가 아닌 한국어를 모델링 할 것이므로, 띄어쓰기가 아닌 별도의 토크나이저를 사용해 토큰화하도록 하겠습니다.

토크나이저는 Kkma, Okt, Komoran 등 다양한 종류가 존재합니다. 가장 정확도가 높은 것은 Kkma이지만, 속도가 가장 빠른 komoran을 여기서는 사용하도록 하겠습니다.

불필요한 품사를 제거하고, 명사, 형용사, 동사, 관형사, 일반부사만 사용하도록 필터링하고, 불용어를 제거하는 함수를 제작합니다.

In [13]:
USER_DICT_PATH = './datasets/user_dic.txt'

custom_dic = [
    '탈원전\tNNG',
]

with open(USER_DICT_PATH, 'w', encoding='utf-8') as f:
    for line in custom_dic:
        f.write(line + '\n')

In [22]:
class CustomTokenizer:

    def __init__(
        self,
        tagger: Callable = Komoran(), # KoNLPy의 태거가 들어갑니다.
        stop_words: Optional[List[str]] = None, # 불용어 사전이 들어갑니다.
        tag_list: List[str] = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'MM', 'MAG']
        ):
      self.tagger = tagger
      self.stop_words = stop_words
      self.tag_list = tag_list

    def __call__(self, text):
      text = text[:10000]
      # 명사, 형용사, 동사, 관형사와 일반 부사만 취한다.
      word_tokens = list(map(lambda x: x[0], filter(lambda x: x[1] in self.tag_list, self.tagger.pos(text))))
      result = [word for word in word_tokens if word not in stop_words]
      return result

In [23]:
STOPWORDS_PATH = './datasets/user_stopwords.csv'

with open(STOPWORDS_PATH, 'r', encoding='cp949') as f:
    tmp = csv.reader(f)
    stop_words = set([l[0] for l in tmp])

tagger = Komoran(userdic='./datasets/user_dic.txt')
tokenizer = CustomTokenizer(tagger = tagger, stop_words = stop_words)
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer)

### **3.3 - 하이퍼파라미터 설정**

**설정해야하는 값**
- `top_n_words` :
    - 토픽(주제) 하나에 들어가는 단어의 수를 의미합니다.
    - 너무 많은 단어가 들어가면, 주제 일관성이 떨어질 수도 있기 때문에 10 ~ 20 사이가 좋습니다.
- `min_topic_size` : (HDBSCAN과 관련)
    - 주제의 최소 크기(주제 안의 문장의 수)를 의미합니다.
    - 이 값이 낮을수록 더 많은 주제가 생성됩니다.
      - 이 값을 너무 낮게 설정하면 마이크로클러스터가 생성됩니다.
      - 이 값을 너무 높게 설정하면 주제가 전혀 생성되지 않을 수도 있습니다.
    - 데이터 세트의 크기에 따라 문서 수가 백만 개에 가까워지면 기본값인 10보다 훨씬 높게 설정하는 것이 좋습니다(예: 100~500).
- `nr_topics` : (HDBSCAN과 관련)
    - 최종적으로 도출할 주제의 수를 의미합니다.
    - 주제가 해당 값에 도달할 때 까지 자동으로 주제의 수를 줄입니다.
    - 'None'으로 설정하면 줄이지 않고, 'auto'로 설정하면 HDBSCAN이 자동으로 줄이게 됩니다.
    - **Tip!** 일단 None으로 하고, 나중에 .reduce_topics로 조절하는걸 추천한다.

In [24]:
top_n_words = 4
min_topic_size = 8
nr_topics = 'auto'

추가적으로 KeyBERT와 같이 MMR을 추가해줄 수 있습니다.

In [29]:
representation_model = MaximalMarginalRelevance(diversity=0.3, top_n_words=top_n_words)

모든 설정을 마쳤습니다. 이제 BERTopic모델을 생성해보도록 하겠습니다.

In [30]:
model = BERTopic(
    top_n_words = top_n_words,
    nr_topics = nr_topics,
    embedding_model = embedding_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    # umap_model=umap_model,
    # hdbscan_model=hdbscan_model,
    # calculate_probabilities = True,
    verbose = True
    )

## **4 - 토픽 추출하기**

이제 키워드를 추출해보겠습니다. 무거운 BERT모델을 사용하므로 꼭 GPU환경에서 실행하셔야하며, 30분 이상의 시간이 소요됩니다.

In [31]:
topics, probs = model.fit_transform(df['news_title'])

2024-02-12 13:56:23,864 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/66 [00:00<?, ?it/s]

2024-02-12 13:56:34,552 - BERTopic - Embedding - Completed ✓
2024-02-12 13:56:34,554 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-12 13:56:45,969 - BERTopic - Dimensionality - Completed ✓
2024-02-12 13:56:45,971 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-12 13:56:46,052 - BERTopic - Cluster - Completed ✓
2024-02-12 13:56:46,054 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-12 13:56:50,773 - BERTopic - Representation - Completed ✓
2024-02-12 13:56:50,780 - BERTopic - Topic reduction - Reducing number of topics
2024-02-12 13:56:56,007 - BERTopic - Topic reduction - Reduced number of topics from 41 to 41


get_topic_info() 메소드를 사용하여 결과를 보겠습니다. 토픽의 개수, 토픽의 크기, 각 토픽에 할당된 단어들을 일부 볼 수 있습니다.

In [32]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,873,-1_탈원전_독일_한전_일방통행,"[탈원전, 독일, 한전, 일방통행]","[에너지 전문가 417명 목소리 경청하길, 한전 탈원전 적자 외면한 전기요금 누진제..."
1,0,84,0_태양광 발전_난개발_벌목_풍력 발전,"[태양광 발전, 난개발, 벌목, 풍력 발전]","[난맥상 드러난 태양광 사업 전면 조사 불가피하다, 태양광 혼란 보고도 풍력발전 확..."
2,1,80,1_월성_수사_감사원_진상,"[월성, 수사, 감사원, 진상]","[월성1호기 원전 폐쇄 재고하라, 멀쩡한 월성1호기 폐쇄 서두르는 이유 뭔가, 월성..."
3,2,71,2_감축_프랑스_탈원전_기후,"[감축, 프랑스, 탈원전, 기후]","[정부 탈원전 폐기 탄소중립 위해 당연한 결정이다, 탄소중립 위한 원전 정책 재검토..."
4,3,70,3_한전_적자_원가_전기료,"[한전, 적자, 원가, 전기료]","[30조 적자 한전이 문재인 공대 에 또 300억 투입 이래도 되나, 한전 적자 부..."
5,4,69,4_공기업_개혁_공공기관 부채_부실,"[공기업, 개혁, 공공기관 부채, 부실]","[공공기관 경영부실 결국 국민 허리만 휜다, 공공기관 부실 정책 부담 떠넘긴 정부 ..."
6,5,60,5_갈등_공사_배_위원회,"[갈등, 공사, 배, 위원회]","[신고리 공론위 발표 뒤가 더 중요하다, 건설 중인 신고리 중단해선 안 된다, 신고..."
7,6,55,6_감사원_최재형_민주당_권력,"[감사원, 최재형, 민주당, 권력]","[정권 비리 덮으려 감사원마저 방탄 기관 만들겠다는 민주당, 민주당 감사원 통제법 ..."
8,7,54,7_경제_위기_쇼크_스태그플레이션,"[경제, 위기, 쇼크, 스태그플레이션]","[부 울 경 의 위기 한국 경제 앞날 예고편일 수 있다, 대기업 신용 강등 경고 주..."
9,8,46,8_탈원전_창원시_송영길_폐지,"[탈원전, 창원시, 송영길, 폐지]","[21조원 원전 수주 탈원전 전면 재고하는 계기 돼야, 대표 소형원전 필요성 또 제..."


## **5 - 후처리**

## **5.1 - 자동으로 토픽 수 줄이기**
결과로 나온 토픽이 너무 많아 줄이고 싶을 수 있습니다. 아래와 같은 코드를 통해 자동으로 설정된 개수만큼 토픽 수를 줄일 수 있습니다.

In [ ]:
# model.reduce_topics(text_list, nr_topics=30)

결과를 보겠습니다.

In [ ]:
model.get_topic_info()

## **5.2 - 수동으로 토픽 수 줄이기**

자동으로 줄이는 것이 불만족스러운 경우, 수동으로 토픽을 임의로 결합할 수 있습니다.

우선, 어떤 토픽들이 연관있는지 보겠습니다.

In [34]:
model.visualize_hierarchy()

관련있는 토픽을 리스트로 구성하고 `merge_topics`에 전달해 실행합니다.

In [49]:
topics_to_merge = [
    [17, 13],
    [6, 2],
    [14, 3],
    [16, 1, 12],
    [15, 4, 5],
    [10],
    [0, 9, 11],
    [8, 18, 7]
    ]
model.merge_topics(df['news_title'], topics_to_merge)

결과를 보겠습니다.

In [ ]:
model.get_topic_info()

## **6 - 시각화**
BERTopic의 장점은 기본적으로 매우 많은 시각화를 지원한다는 것입니다. 여러가지 시각화를 같이 진행해보도록 하겠습니다.

### **6.1 - 일반 시각화**

In [35]:
model.visualize_topics()

In [36]:
model.visualize_barchart(top_n_topics=15)

In [37]:
model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

### **6.2 - 시계열 시각화**

tiemstamp를 넣어주면, 시기별로 토픽이 어떻게 변했는지도 시각화할 수 있습니다.

In [38]:
timestamps = df['news_date'].to_list()

topics_over_time = model.topics_over_time(
    docs=df['news_title'],
    timestamps=timestamps,
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=30
)

model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

30it [00:25,  1.15it/s]


## **수고하셨습니다**

더 많은 사용법은 [공식 GitHub](https://maartengr.github.io/BERTopic/index.html#installation)에서 찾아보세요!